In [13]:
import random
import pandas

In [14]:
%store -r

In [15]:
len(unique_rejected_clean_training), len(unique_accepted_clean_training_abstracts)

(590, 1179)

In [16]:
random.shuffle(unique_accepted_clean_training_abstracts)

In [17]:
unique_accepted_clean_training_abstracts_short = unique_accepted_clean_training_abstracts[0:590]

In [18]:
len(unique_accepted_clean_training_abstracts_short)

590

In [19]:
# Combine our data into two lists, one with abstracts, and the other with the labels. 
labels_clean_training = []
abstracts_clean_training = [] 
for abstract in unique_accepted_clean_training_abstracts_short:
    abstracts_clean_training.append(abstract)
    labels_clean_training.append("selected")
for abstract in unique_rejected_clean_training:
    abstracts_clean_training.append(abstract)
    labels_clean_training.append("rejected")

In [20]:
%store abstracts_clean_training

Stored 'abstracts_clean_training' (list)


In [21]:
%store labels_clean_training

Stored 'labels_clean_training' (list)


In [22]:
# create the data frame
trainCleanBalancedDF = pandas.DataFrame()
trainCleanBalancedDF['text'] = abstracts_clean_training
trainCleanBalancedDF['label'] = labels_clean_training

In [23]:
trainCleanBalancedDF[0:3]

,text,label
0,Background:The posterior cruciate ligament (PC...,selected
1,Background: An initial and vital important ste...,selected
2,Objectives: Reliable epidemiologic estimates o...,selected


In [143]:
# now we do the model creation

In [144]:
# Features and Labels
X = trainCleanBalancedDF['text']
ylabels = trainCleanBalancedDF['label']
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.25, random_state=42)

In [145]:
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens

# show the core model

In [146]:
# Vectorization
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
classifier = LinearSVC()
# classifier = SVC(probability=True, class_weight="balanced",C=0.1, gamma= 0.01)
# classifier = SVC(probability=True, 
#                  class_weight="balanced",
#                  C=0.01, 
#                  gamma= 0.1
#                 )
# Using Tfidf
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [147]:
# for out clearner, vectorize and classifier defined earlier, we are using the full abstract texxt here. 
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfvectorizer),
                 ('classifier', classifier)])

In [148]:
# fit the data
# Fit our data
pipe.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('cleaner', <__main__.predictors object at 0x1c66e2e3c8>), ('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ng...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [149]:
# get predictions
y_pred = pipe.predict(X_test)

In [128]:
# get pred_proba as we are using 
# classifier = SVC(probability=True, class_weight="balanced",C=0.1, gamma= 0.01) and not Linear SVC 
ye_pred_prob = pipe.predict_proba(X_test)

# Show how poor the model preforms 

In [150]:
# look at the preformacne of this model
metrics.confusion_matrix(y_pred, y_test), metrics.accuracy_score(y_test, y_pred)

(array([[83, 53],
        [65, 94]]), 0.6)